In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy
import sys
import time

In [2]:
class Graph:
    
    def __init__(self, n):
        self.n_nodes = n
        self.has_matrix = False
        self.matrix_error_size = None
        try:
            self.matrix = np.zeros((n, n), dtype="uint8")
            self.has_matrix = True
        except MemoryError as error:
            print("Cannot create matrix >>", error)
            ea = str(error)
            eb = ea[ea.index("iB for")-1]
            eb_m = 10**3 if eb=="G" else 10**6
            self.matrix_error_size = float(ea[ea.index("allocate")+9:ea.index("iB for")-2]) * eb_m
        self.nodes = [set() for i in range(n)]
    
    
    def add_edge(self, v, w):
        
        if v == w:
            if self.has_matrix:
                self.matrix[v-1, v-1] = 2
            self.nodes[v-1].add(v)
            
        else:
            if self.has_matrix:
                self.matrix[v-1, w-1] = 1
                self.matrix[w-1, v-1] = 1
            self.nodes[v-1].add(w)
            self.nodes[w-1].add(v)
    
    def get_node(self, v):
        return self.nodes[v-1]
    
    def get_lists(self):
        return self.nodes
    
    def get_node_edges(self):
        return {i+1:self.nodes[i] for i in range(self.n_nodes)}
    
    def get_matrix(self):
        if self.has_matrix:
            return self.matrix
        return None
    
    def get_matrix_beautiful(self):
        if self.has_matrix:
            return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
        return None
    
    def sort_neighbors(self):
        self.nodes = [sorted(n) for n in self.nodes]

In [3]:
def open_graph_txt(filename, extra=False):
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        n_nodes = int(lines[0])
        edges = [tuple(map(lambda i: int(i), line.split(" "))) for line in lines[1:-1]]
    
    graph = Graph(n_nodes)
    for v, w in edges:
        graph.add_edge(v, w)
    
    if extra:
        return graph, n_nodes, edges
    
    return graph

In [4]:
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    if graph.has_matrix:
        print("Número de arestas:", graph.get_matrix().sum()/2)
        print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
        print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
        print("Grau médio:", graph.get_matrix().sum(axis=0).mean())
        print("Mediana do Grau:", np.median(graph.get_matrix().sum(axis=0)))
    else:
        print("Número de arestas:", np.sum([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())])/2)
        print("Grau mínimo:", np.min([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
        print("Grau máximo:", np.max([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
        print("Grau médio:", np.mean([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
        print("Mediana do Grau:", np.median([len(x) if i+1 not in x else len(x) + 1 for i, x in enumerate(graph.get_lists())]))
    print("List: ", sys.getsizeof(graph.get_lists())/(10**6), "MB")
    print("Matrix: ", (str(sys.getsizeof(graph.get_matrix())/(10**6))) if graph.has_matrix else (graph.matrix_error_size), "MB")

In [5]:
class DFSL:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.level[root-1] = 0
        
        self.start_root(root)
    
    def start_root(self, root):
        self.stack = []
        self.stack.append(root)
    
    def search(self):
        while(len(self.stack) != 0):
            u = self.stack.pop()
            
            if not self.visited[u-1]:
                self.visited[u-1] = 1
                
                for v in self.graph.nodes[u-1][::-1]:
                    if not self.visited[v-1]:
                        self.stack.append(v)
                        self.parent[v-1] = u
                        self.level[v-1] = self.level[u-1] + 1
                        

In [6]:
class DFSM:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.level[root-1] = 0
        
        self.start_root(root)
    
    def start_root(self, root):
        self.stack = []
        self.stack.append(root)
    
    def search(self):
        while(len(self.stack) != 0):
            u = self.stack.pop()
            
            if not self.visited[u-1]:
                self.visited[u-1] = 1
                
                for v in np.flip(np.argwhere(self.graph.matrix[u-1, :] == 1).reshape(-1)) + 1:
                    if not self.visited[v-1]:
                        self.stack.append(v)
                        self.parent[v-1] = u
                        self.level[v-1] = self.level[u-1] + 1
                        

In [7]:
class BFSL:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        
        self.level[root-1] = 0
        self.visited[root-1] = 1
        
        self.start_root(root)
        
    def start_root(self, root):
        self.queue = []
        self.queue.append(root)
        
    def search(self):
        
        while(len(self.queue)):
            v = self.queue.pop(0)
            
            for w in self.graph.nodes[v-1]:
                if v == w:
                        continue
                if not self.visited[w-1]:
                    self.visited[w-1] = 1
                    self.queue.append(w)
                    self.parent[w-1] = v
                    self.level[w-1] = self.level[v-1] + 1

In [8]:
class BFSM:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        
        self.level[root-1] = 0
        self.visited[root-1] = 1
        
        self.start_root(root)
        
    def start_root(self, root):
        self.queue = []
        self.queue.append(root)
        
    def search(self):
        
        while(len(self.queue)):
            v = self.queue.pop(0)
            
            for w in np.argwhere(self.graph.matrix[v-1, :] == 1).reshape(-1) + 1:
                if v == w:
                        continue
                if not self.visited[w-1]:
                    self.visited[w-1] = 1
                    self.queue.append(w)
                    self.parent[w-1] = v
                    self.level[w-1] = self.level[v-1] + 1

In [9]:
class MinimumPath:
    
    def __init__(self, graph):
        self.graph = graph
        self.matrix = np.full((graph.n_nodes, graph.n_nodes), fill_value=-1, dtype="int32")
        self.run()
    
    def run(self):
        for v in tqdm(range(1, self.graph.n_nodes+1)):
            bfsl = BFSL(self.graph, v)
            bfsl.search()
            for bfsl_node_index in np.argwhere(bfsl.visited == 1).reshape(-1):
                self.matrix[v-1, bfsl_node_index] = bfsl.level[bfsl_node_index]
            del bfsl
    
    def get_distance(self, u, v):
        return self.matrix[u-1, v-1]
    
    def get_diameter(self):
        return np.max(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.graph.n_nodes+1), index=np.arange(1, self.graph.n_nodes+1))
    

In [10]:
class Components:
    
    def __init__(self, graph):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.components = []
        
        while np.argwhere(self.visited == 0).reshape(-1).shape[0] > 0:
            root = np.argwhere(self.visited == 0).reshape(-1)[0] + 1

            bfsl = BFSL(self.graph, root)
            bfsl.search()
            
            bfsl_visited_index = np.argwhere(bfsl.visited == 1).reshape(-1)
            
            self.visited[bfsl_visited_index] = 1
            self.components.append((bfsl_visited_index+1).tolist())

    def get_components(self):
        a = sorted(self.components, key=lambda x: len(x), reverse=True)
        b = [len(x) for x in a]
        c = list(zip(b, a))
        return c


In [11]:
# from pyvis.network import Network

# net = Network(notebook=True)

# for v in range(1, graph.n_nodes+1):
#     net.add_node(v, label=v)
    
# for v, neighbors in enumerate(graph.nodes):
#     for w in neighbors:
#         net.add_edge(v+1, w)

# net.show("graph.html")

In [12]:
folder = "inputs"
filename = "grafo_5.txt"

path = os.path.join(folder, filename)

graph = open_graph_txt(path)
graph.sort_neighbors()

# print(graph.has_matrix)
# graph_statistics(graph)
# graph.has_matrix = False
# print("----")
# print(graph.has_matrix)
graph_statistics(graph)

Cannot create matrix >> Unable to allocate 931. GiB for an array with shape (1000022, 1000022) and data type uint8
Número de vértices: 1000022
Número de arestas: 17666161.0
Grau mínimo: 1
Grau máximo: 87
Grau médio: 35.331544706016466
Mediana do Grau: 37.0
List:  8.448728 MB
Matrix:  931000.0 MB


In [13]:
dfsl = DFSL(graph, 1)
dfsl.search()

print("DFSL Result:")
dfsl_df = pd.DataFrame(list(zip(range(1, dfsl.graph.n_nodes+1), dfsl.level, dfsl.parent)), columns=["node", "level", "parent"], index=np.arange(1, dfsl.graph.n_nodes+1))
dfsl_df.to_csv("./outputs/dfsl_test.csv")

KeyboardInterrupt: 

In [ ]:
dfsm = DFSM(graph, 1)
dfsm.search()

print("DFSM Result:")
dfsm_df = pd.DataFrame(list(zip(range(1, dfsm.graph.n_nodes+1), dfsm.level, dfsm.parent)), columns=["node", "level", "parent"], index=np.arange(1, dfsm.graph.n_nodes+1))
dfsm_df.to_csv("./outputs/dfsm_test.csv")

In [ ]:
bfsl = BFSL(graph, 1)
bfsl.search()

print("BFSL Result:")
bfsl_df = pd.DataFrame(list(zip(range(1, bfsl.graph.n_nodes+1), bfsl.level, bfsl.parent)), columns=["node", "level", "parent"], index=np.arange(1, bfsl.graph.n_nodes+1))
bfsl_df.to_csv("./outputs/bfsl_test.csv")

In [ ]:
bfsm = BFSM(graph, 1)
bfsm.search()

print("BFSM Result:")
bfsm_df = pd.DataFrame(list(zip(range(1, bfsm.graph.n_nodes+1), bfsm.level, bfsm.parent)), columns=["node", "level", "parent"], index=np.arange(1, bfsm.graph.n_nodes+1))
bfsm_df.to_csv("./outputs/bfsm_test.csv")

In [ ]:
minpath = MinimumPath(graph)

print("Minpath Distance:", minpath.get_distance(1, 3))
print("Minpath Diameter:", minpath.get_diameter())

In [ ]:
components = Components(graph)

with open("./outputs/components_test.txt", mode="w") as out:
    for line in components.get_components():
        out.write(str(line))
        out.write("\n")